In [1]:
from huggingface_hub import notebook_login, interpreter_login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token can be pasted using 'Right-Click'.


Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  n


Token is valid (permission: read).
Your token has been saved to C:\Users\katko\.cache\huggingface\token
Login successful


In [2]:
from transformers.utils import send_example_telemetry

send_example_telemetry("token_classification_notebook", framework="pytorch")

In [3]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## Preprocessing

In [4]:
from preprocessing import extract_data_from_conll_extended
extract_data_from_conll_extended("data/en_ewt-up-test.conllu", "dummy_train.json")

In [5]:
from datasets import load_dataset, load_metric
datasets = load_dataset('json', data_files={'train': 'dummy_train.json', 'test': 'dummy_test.json'})
datasets

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['predicate', 'predicate_position', 'arguments', 'word'],
        num_rows: 4799
    })
    test: Dataset({
        features: ['predicate', 'predicate_position', 'arguments', 'word'],
        num_rows: 12
    })
})

In [6]:
datasets["train"][0]

{'predicate': 'morph.01',
 'predicate_position': 4,
 'arguments': ['_', '_', 'ARG1', 'V', '_', 'ARG2', '_'],
 'word': ['What', 'if', 'Google', 'Morphed', 'Into', 'GoogleOS', '?']}

In [7]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(datasets["train"])

,predicate,predicate_position,arguments,word
0,point_out.02,3,"[_, ARG0, V, _, _, _, _, ARG1, _, _, _, _, _, _, _, _, _, _, _]","[One, reader, pointed, out, that, the, President, watched, the, Americans, in, the, embassy, when, they, took, them, hostage, .]"
1,send.01,15,"[_, _, _, _, _, _, _, _, _, _, _, _, _, _, V, _, _, _, _, ARG1, _, _, _, _, _, _, _, _, _, _, _, _, _]","[I, just, wanted, to, follow, up, on, whether, you, will, have, a, chance, to, send, a, draft, Credit, Support, Annex, (, similar, in, form, to, the, one, previously, executed, with, ENA, ), .]"
2,use.01,3,"[ARGM-MOD, ARGM-NEG, V, ARGM-TMP, _]","[Will, never, use, again, .]"
3,say.01,7,"[_, _, ARGM-CAU, ARG0, _, ARGM-NEG, V, _, _, _, ARG1, _, _, _, _]","[Which, is, why, he, did, n't, say, we, 're, at, war, with, Islamic, people, .]"
4,ensure.01,9,"[_, _, _, _, _, _, _, _, V, _, ARG1, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _]","[Airfare, alone, will, be, incredibly, expensive, so, make, sure, you, have, the, money, and, of, course, free, time, to, take, your, time, and, have, a, great, time, .]"
5,be.02,5,"[_, _, _, _, V, _, _, ARG1, _, _, _]","[i, always, thought, there, s, no, custom, charges, for, gifts, .]"
6,do.02,7,"[_, _, _, ARG1, ARG0, _, V, _, _, _, _, _, _, _, _, _, _]","[These, guys, know, what, they, 're, doing, and, helped, me, in, all, phases, of, our, project, .]"
7,take.01,12,"[_, _, _, _, _, _, _, _, _, ARG0, _, V, ARG1, _, _, _, ARGM-MNR, _, _, _, ARGM-LOC, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _]","[In, a, communique, ,, the, AMS, said, that, for, Iraqis, to, take, part, with, "", raiding, forces, "", in, the, assault, on, a, city, ,, the, population, of, which, is, Muslim, (, such, as, Fallujah, ), would, be, considered, the, most, mortal, of, mortal, sins, .]"
8,worth.02,3,"[_, _, V, _, ARG1, _, _, _, _, _]","[is, it, worthy, to, send, in, for, warranty, repair, ?]"
9,have.01,3,"[_, _, V, _, _, _, _, _, _, _, _, _, _, _, _]","[Now, I, 've, found, someone, who, can, manage, to, do, what, I, really, want, .]"


In [9]:
datasets["train"][0]

{'predicate': 'morph.01',
 'predicate_position': 4,
 'arguments': ['_', '_', 'ARG1', 'V', '_', 'ARG2', '_'],
 'word': ['What', 'if', 'Google', 'Morphed', 'Into', 'GoogleOS', '?']}

### Tokenization

In [10]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [12]:
def tokenize(instance):
    def align(tokenized_input, labels):
        word_ids = tokenized_input.word_ids()
        aligned_labels = ['_'] * len(word_ids) 
        label_index = 0 
        for i, word_id in enumerate(word_ids):
            try:
                if word_id is None:
                    aligned_labels[i] = '[PAD]'
                    continue 
                original_label = labels[word_id]
                if original_label == '_':
                    continue 
                if i == 0 or word_id != word_ids[i-1]:
                    prefix = ''
                else:
                    prefix = ''
                aligned_labels[i] = f'{prefix}{original_label}'
            except:
                pass
    
        return aligned_labels
    sentence = instance["word"]
    labels = instance["arguments"]
    tokenized_input = tokenizer(sentence, is_split_into_words=True)
    predicate = instance['predicate']
    predicate = tokenizer(predicate)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
    aligned_labels = align(tokenized_input, labels)
    tokenized_input['input_ids'].extend(predicate['input_ids'][1:]), tokenized_input['attention_mask'].extend(predicate['attention_mask'][1:]), aligned_labels.extend(['[PAD]' for i in range(len(predicate['input_ids'][1:]))])
    tokenized_input['labels'] = aligned_labels
    
    return tokenized_input


In [13]:
print(len(datasets['train']))
training_set = [tokenize(instance) for instance in datasets['train']]


4799


In [14]:
sample = training_set[0]
print(len(training_set))
print(len(sample))

4799
3


In [15]:
from datasets import Dataset, Features, Sequence, Value

features = Features({
    "input_ids": Sequence(feature=Value(dtype='int64')),
    "attention_mask": Sequence(feature=Value(dtype='int64')),
    "labels": Sequence(feature=Value(dtype='string')),
})

In [16]:
dataset_train = Dataset.from_dict({"input_ids": [item['input_ids'] for item in training_set],
                             "attention_mask": [item['attention_mask'] for item in training_set],
                             "labels": [item['labels'] for item in training_set]}, features=features)

In [17]:
print(tokenized_input.word_ids())


NameError: name 'tokenized_input' is not defined

In [18]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
label_list = set(label for instance in datasets["train"] for label in instance['arguments'])

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [20]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [21]:
metric = load_metric("seqeval")
metric = load_metric("seqeval")


C:\Users\katko\AppData\Local\Temp\ipykernel_12500\1988684555.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
C:\Users\katko\miniconda3\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
C:\Users\katko\miniconda3\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load

In [22]:
labels = label_list
metric.compute(predictions=[labels], references=[labels])

C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ARGM-PRP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: R-ARGM-LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ARGM-EXT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: C-ARG1-DSP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: R-ARGM-DIR seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site

{'ADJ': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ADV': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARG0': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARG1': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARG1-DSP': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARG2': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARG3': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARGM-ADJ': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARGM-ADV': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARGM-CXN': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARGM-DIR': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARGM-LOC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARGM-MNR': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARGM-TMP': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'CAU': {'precisio

In [23]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [24]:
trainer = Trainer(
    model,
    args,
    train_dataset=dataset_train,
    eval_dataset=dataset_train,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-65e35587-15a004146aa7a9e5746ff163;1584d25f-154d-43f1-82e9-dff5fda5b9ad)

Repository Not Found for url: https://huggingface.co/api/models/distilbert-base-uncased-finetuned-ner.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.

In [ ]:
trainer.train()